In [ ]:
import pyodbc
from secrets import CONNECTION_STRING, SQL_ALCHEMY

conn = pyodbc.connect(CONNECTION_STRING, autocommit=True)

table_name = 'MatchData'

create_table_sql = f"""CREATE TABLE {table_name} (
    MATCHDATE date,
    FTR varchar(2),
    HTP float,
    ATP float,
    HM1_L float,
    HM1_D float,
    HM1_W float,
    HM1_M float,
    HM2_L float,
    HM2_D float,
    HM2_W float,
    HM2_M float,
    HM3_L float,
    HM3_D float,
    HM3_W float,
    HM3_M float,
    AM1_L float,
    AM1_D float,
    AM1_W float,
    AM1_M float,
    AM2_L float,
    AM2_D float,
    AM2_W float,
    AM2_M float,
    AM3_L float,
    AM3_D float,
    AM3_W float,
    AM3_M float,
    HTGD float,
    ATGD float,
    DiffFormPts float,
    HomeTeamName varchar(50),
    AwayTeamName varchar(50)
);"""

cursor = conn.cursor()
cursor.execute(create_table_sql)

cursor.commit()

cursor.close()

In [ ]:
import sqlalchemy
import pyodbc
import pandas as pd

engine = sqlalchemy.create_engine(SQL_ALCHEMY, echo=False)


df = pd.read_csv('PL_Match_Data.csv')
df.to_sql('MatchData', con=engine, if_exists='append', index=False)

conn.close()

In [ ]:
import requests

matches_api = 'https://livescore-api.com/api-client/scores/history.json?competition_id=2&key=oE0Ap0OZQu7rMokc&secret=tLFH3Vht7xda02voHvZ6wN6mhdN4C7h5&from=2022-08-05&to=2023-05-28'
response1 = requests.get(matches_api)
response1.json()['data']['match']

In [ ]:
response1.json()['data']['match'][0]

fixture_id = response1.json()['data']['match'][0]['id']

api_lineups = f'https://livescore-api.com/api-client/matches/lineups.json?match_id={fixture_id}&key=oE0Ap0OZQu7rMokc&secret=tLFH3Vht7xda02voHvZ6wN6mhdN4C7h5'

response2 = requests.get(api_lineups)

In [ ]:
response2.json()['data']['lineup']['home']['players']

In [ ]:
import pandas as pd
import requests

columns = ['Date', 'HomeTeam', 'AwayTeam', 'HomeLineup', 'AwayLineup']
lineup_data = pd.DataFrame(columns=columns)

for i in range(1, 8):
    next_link = f'https://livescore-api.com/api-client/scores/history.json?key=oE0Ap0OZQu7rMokc&to=2023-05-28&competition_id=2&from=2022-08-05&secret=tLFH3Vht7xda02voHvZ6wN6mhdN4C7h5&page={i}'
    response1 = requests.get(next_link)
    for j in range(len(response1.json()['data']['match'])):
        match_id = response1.json()['data']['match'][j]['id']
        api_lineups = f'https://livescore-api.com/api-client/matches/lineups.json?match_id={match_id}&key=oE0Ap0OZQu7rMokc&secret=tLFH3Vht7xda02voHvZ6wN6mhdN4C7h5'
        response2 = requests.get(api_lineups)
        new_data = {'Date': response1.json()['data']['match'][j]['date'], 
                    'HomeTeam': response1.json()['data']['match'][j]['home_name'],
                    'AwayTeam': response1.json()['data']['match'][j]['away_name'],
                    'HomeLineup': str([d['name'] for d in response2.json()['data']['lineup']['home']['players'][0:11]]),
                    'AwayLineup': str([d['name'] for d in response2.json()['data']['lineup']['away']['players'][0:11]])}
        lineup_data = pd.concat([lineup_data, pd.DataFrame([new_data])], ignore_index=True)

lineup_data.to_csv('PL_Lineup_Data2.csv', index=False)

In [ ]:
id_matches = [response1.json()['data']['match'][j]['id'] for j in range(len(response1.json()['data']['match']))]
id_matches

api_lineups = f''
response2 = requests.get(api_lineups)
new_data = {'Date': response1.json()['data']['match'][j]['date'], 
                    'HomeTeam': response1.json()['data']['match'][j]['home_name'],
                    'AwayTeam': response1.json()['data']['match'][j]['away_name'],
                    'HomeLineup': str([d['name'] for d in response2.json()['data']['lineup']['home']['players'][0:11]]),
                    'AwayLineup': str([d['name'] for d in response2.json()['data']['lineup']['away']['players'][0:11]])}

new_data

In [ ]:
import numpy as np

dataframe_new = pd.read_csv('PL_Lineup_Data2.csv')

dataframe_new['HomeTeam'] = np.where(dataframe_new['HomeTeam'].astype(str).str[:3] != 'AFC',
                                     dataframe_new['HomeTeam'].astype(str) + ' FC',
                                     dataframe_new['HomeTeam'].astype(str))

dataframe_new['AwayTeam'] = np.where(dataframe_new['AwayTeam'].astype(str).str[:3] != 'AFC',
                                     dataframe_new['AwayTeam'].astype(str) + ' FC',
                                     dataframe_new['AwayTeam'].astype(str))

dataframe_new

In [ ]:
conn = pyodbc.connect(CONNECTION_STRING, autocommit=True)

table_name = 'LineupData'

create_table_sql = f"""CREATE TABLE {table_name} (
                        Date datetime,
                        HomeTeam varchar(255),
                        AwayTeam varchar(255),
                        HomeLineup varchar(255),
                        AwayLineup varchar(255)
                        )"""
                        
cursor = conn.cursor()
cursor.execute(create_table_sql)
cursor.commit()

engine = sqlalchemy.create_engine(SQL_ALCHEMY, echo=False)

dataframe_new.to_sql('LineupData', con=engine, if_exists='append', index=False)

conn.close()

In [ ]:
import numpy as np
import pandas as pd 
import pyodbc
import sqlalchemy 

dataframe_new = pd.read_csv('PL_Lineup_Data2.csv')

dataframe_new['HomeTeam'] = np.where(dataframe_new['HomeTeam'].astype(str).str[:3] != 'AFC',
                                     dataframe_new['HomeTeam'].astype(str) + ' FC',
                                     dataframe_new['HomeTeam'].astype(str))

dataframe_new['AwayTeam'] = np.where(dataframe_new['AwayTeam'].astype(str).str[:3] != 'AFC',
                                     dataframe_new['AwayTeam'].astype(str) + ' FC',
                                     dataframe_new['AwayTeam'].astype(str))

engine = sqlalchemy.create_engine(SQL_ALCHEMY, echo=False)

dataframe_new.to_sql('LineupData', con=engine, if_exists='append', index=False)